In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.contrib.keras.api.keras.backend as K
from tensorflow.python.keras.utils import multi_gpu_model
from tensorflow.python.keras.models import Model, Sequential, model_from_config
from tensorflow.python.keras.layers import Input , Reshape, Permute, Lambda
from tensorflow.python.keras.layers import Add, add, multiply
from tensorflow.python.keras.layers import Conv2D, Conv2DTranspose
from tensorflow.python.keras.layers import Conv3D, Conv3DTranspose
from tensorflow.python.keras.layers import Activation, Dropout, BatchNormalization
from tensorflow.python.keras.layers import concatenate, MaxPooling2D
from tensorflow.python.keras.optimizers import RMSprop, Adam

def CostVolume(inputs, max_d):
        left_tensor, right_tensor = inputs
        shape = right_tensor.shape
        right_tensor = K.spatial_2d_padding(right_tensor, padding=((0, 0), (max_d, 0)))
        disparity_costs = []
        for d in reversed(range(max_d)):
            left_tensor_slice = left_tensor
            right_tensor_slice = tf.slice(right_tensor, begin = [0, 0, d, 0], size = [-1, -1, shape[2], -1])
            cost = K.concatenate([left_tensor_slice, right_tensor_slice], axis = 3)
            disparity_costs.append(cost)
        cost_volume = K.stack(disparity_costs, axis = 1)
        return cost_volume
    
def SoftArgMax(inputs, height, width, disp_range):
        print('1')
        tmp = tf.squeeze(inputs, squeeze_dims=-1)
        print('2')
        softmax = tf.nn.softmax(-tmp)
        print('3')
        disp_mat = tf.constant(list(map(lambda x: x, range(1, 192+1, 1))), shape=(192, 512, 960))
        disp_mat = tf.cast(disp_mat, tf.float32)
        print('4')
        result = tf.multiply(softmax, disp_mat)
        print('5')
        result = tf.reduce_sum(result, axis = 1)
        return result

In [ ]:
input_shape = (512, 960, 3)
img = Input(shape = input_shape, dtype = "float32", name='l_img')
layer1 = Conv2D(32, (5, 5), (2, 2), padding='same', activation='relu', name='layer1')(img)
layer2 = Conv2D(32, (3, 3), (1, 1), padding='same', activation='relu', name='layer2')(layer1)
layer3 = Conv2D(32, (3, 3), (1, 1), padding='same', activation='relu', name='layer3')(layer2)        
short_cut1 = add([layer1, layer3])

layer4 = Conv2D(32, (3, 3), (1, 1), padding='same', activation='relu', name='layer4')(short_cut1)
layer5 = Conv2D(32, (3, 3), (1, 1), padding='same', activation='relu', name='layer5')(layer4)
short_cut2 = add([layer4, layer5])
layer6 = Conv2D(32, (3, 3), (1, 1), padding='same', activation='relu', name='layer6')(short_cut2)
layer7 = Conv2D(32, (3, 3), (1, 1), padding='same', activation='relu', name='layer7')(layer6)
short_cut3 = add([layer6, layer7])
layer8 = Conv2D(32, (3, 3), (1, 1), padding='same', activation='relu', name='layer8')(short_cut3)
layer9 = Conv2D(32, (3, 3), (1, 1), padding='same', activation='relu', name='layer9')(layer8)
short_cut4 = add([layer8, layer9])
layer10 = Conv2D(32, (3, 3), (1, 1), padding='same', activation='relu', name='layer10')(short_cut4)
layer11 = Conv2D(32, (3, 3), (1, 1), padding='same', activation='relu', name='layer11')(layer10)
short_cut5 = add([layer10, layer11])
layer12 = Conv2D(32, (3, 3), (1, 1), padding='same', activation='relu', name='layer12')(short_cut5)
layer13 = Conv2D(32, (3, 3), (1, 1), padding='same', activation='relu', name='layer13')(layer12)
short_cut6 = add([layer12, layer13])
layer14 = Conv2D(32, (3, 3), (1, 1), padding='same', activation='relu', name='layer14')(short_cut6)
layer15 = Conv2D(32, (3, 3), (1, 1), padding='same', activation='relu', name='layer15')(layer14)
short_cut7 = add([layer14, layer15])
layer16 = Conv2D(32, (3, 3), (1, 1), padding='same', activation='relu', name='layer16')(short_cut7)
layer17 = Conv2D(32, (3, 3), (1, 1), padding='same', activation='relu', name='layer17')(layer16)
short_cut8 = add([layer16, layer17])
       
layer18 = Conv2D(32, (3, 3), (1, 1), padding='same', name='layer18')(short_cut8)

#net = Model(inputs = img, outputs = layer18)
#net.summary() 

In [ ]:
unifeatures = [layer18, layer18]   
cv_l = Lambda(CostVolume, arguments = {'max_d':(int)(192 / 2)}, name='CostVolume_left')(unifeatures)

layer19 = Conv3D(32, (3, 3, 3), (1, 1, 1), padding='same', activation='relu')(cv_l)
layer20 = Conv3D(32, (3, 3, 3), (1, 1, 1), padding='same', activation='relu')(layer19)
layer21 = Conv3D(64, (3, 3, 3), (2, 2, 2), padding='same', activation='relu')(layer20)
layer22 = Conv3D(32, (3, 3, 3), (1, 1, 1), padding='same', activation='relu')(layer21)
layer23 = Conv3D(32, (3, 3, 3), (1, 1, 1), padding='same', activation='relu')(layer22)
layer24 = Conv3D(64, (3, 3, 3), (2, 2, 2), padding='same', activation='relu')(layer23)
layer25 = Conv3D(32, (3, 3, 3), (1, 1, 1), padding='same', activation='relu')(layer24)
layer26 = Conv3D(32, (3, 3, 3), (1, 1, 1), padding='same', activation='relu')(layer25)
layer27 = Conv3D(64, (3, 3, 3), (2, 2, 2), padding='same', activation='relu')(layer26)
layer28 = Conv3D(32, (3, 3, 3), (1, 1, 1), padding='same', activation='relu')(layer27)
layer29 = Conv3D(32, (3, 3, 3), (1, 1, 1), padding='same', activation='relu')(layer28)
layer30 = Conv3D(64, (3, 3, 3), (2, 2, 2), padding='same', activation='relu')(layer29)
layer31 = Conv3D(32, (3, 3, 3), (1, 1, 1), padding='same', activation='relu')(layer30)
layer32 = Conv3D(32, (3, 3, 3), (1, 1, 1), padding='same', activation='relu')(layer31)

layer33 = Conv3DTranspose(64, (3, 3, 3), (2, 2, 2), padding='same', activation='relu')(layer32)
short_cut1 = add([layer27, layer33])
layer34 = Conv3DTranspose(64, (3, 3, 3), (2, 2, 2), padding='same', activation='relu')(short_cut1)
short_cut2 = add([layer24, layer34])
layer35 = Conv3DTranspose(64, (3, 3, 3), (2, 2, 2), padding='same', activation='relu')(short_cut2)
short_cut3 = add([layer21, layer35])
layer36 = Conv3DTranspose(32, (3, 3, 3), (2, 2, 2), padding='same', activation='relu')(short_cut3)
short_cut4 = add([layer20, layer36])
layer37 = Conv3DTranspose(1, (3, 3, 3), (2, 2, 2), padding='same')(short_cut4)
print('############################')
disp_map = Lambda(SoftArgMax, 
                  arguments = {'height':512, 'width':960, 'disp_range':192}, 
                 )(layer37)

net = Model(inputs = img, outputs = disp_map)
net.summary() 

from tensorflow.python.keras.utils import plot_model

plot_model(net, to_file='test_model.png')

In [ ]:
import tensorflow as tf

#list(map(lambda x: x, range(1, 192+1, 1)))
disp_mat = tf.constant(list(map(lambda x: x, range(1, 192+1, 1))), shape=(192, 512, 960))
sess = tf.Session()
result = sess.run(disp_mat)

print(result)